In [10]:
import pandas as pd
import nltk; nltk.download('stopwords')
nltk.download('punkt')

from nltk.tokenize import word_tokenize 

# Regular Expressions
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
# import logging
# logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

# import warnings
# warnings.filterwarnings("ignore",category=DeprecationWarning)

# from ngram import NGram

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/tristandealwis/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/tristandealwis/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [11]:
listings = pd.read_csv("listings.csv", usecols=['id', 'name', 'description', 'neighbourhood_cleansed', 'zipcode', 'property_type', 'room_type', 
                                            'price', 'availability_365'])
listings = listings.rename(columns={'id': 'listing_id'})

reviews = pd.read_csv("reviews.csv", usecols=['listing_id', 'comments'])

In [12]:
df = pd.merge(listings, reviews, on='listing_id')

In [13]:
df.describe()

,listing_id,availability_365
count,1.166689e+06,1.166689e+06
mean,1.296548e+07,1.628377e+02
std,1.005105e+07,1.247375e+02
min,3.831000e+03,0.000000e+00
25%,3.768977e+06,4.000000e+01
50%,1.199379e+07,1.500000e+02
75%,2.070862e+07,2.860000e+02
max,3.855316e+07,3.650000e+02


In [14]:
df.head(5)

,listing_id,name,description,neighbourhood_cleansed,zipcode,property_type,room_type,price,availability_365,comments
0,3831,Cozy Entire Floor of Brownstone,Urban retreat: enjoy 500 s.f. floor in 1899 br...,Clinton Hill,11238,Guest suite,Entire home/apt,$89.00,192,"lisa is a wonderful, kind and thoughtful host...."
1,3831,Cozy Entire Floor of Brownstone,Urban retreat: enjoy 500 s.f. floor in 1899 br...,Clinton Hill,11238,Guest suite,Entire home/apt,$89.00,192,The best thing about this place is the locatio...
2,3831,Cozy Entire Floor of Brownstone,Urban retreat: enjoy 500 s.f. floor in 1899 br...,Clinton Hill,11238,Guest suite,Entire home/apt,$89.00,192,LisaRoxanne was friendly and helpful. The room...
3,3831,Cozy Entire Floor of Brownstone,Urban retreat: enjoy 500 s.f. floor in 1899 br...,Clinton Hill,11238,Guest suite,Entire home/apt,$89.00,192,LisaRoxanne is very nice and communicative. Th...
4,3831,Cozy Entire Floor of Brownstone,Urban retreat: enjoy 500 s.f. floor in 1899 br...,Clinton Hill,11238,Guest suite,Entire home/apt,$89.00,192,Lisa was a very pleasant and nice host that ac...


In [16]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [36]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(df['comments'][1:100000]))

In [37]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['the', 'best', 'thing', 'about', 'this', 'place', 'is', 'the', 'location', 'you', 're', 'right', 'in', 'the', 'middle', 'of', 'clinton_hill', 'with', 'easy', 'access', 'to', 'restaurants', 'on', 'franklin_avenue', 'and', 'in', 'fort_greene', 'you', 're', 'also', 'close', 'to', 'the', 'and', 'trains', 'so', 'transportation', 'is', 'easy', 'the', 'space', 'is', 'clean', 'and', 'neat', 'if', 'spare', 'it', 'really', 'is', 'much', 'more', 'of', 'one', 'floor', 'apartment', 'than', 'private', 'room', 'as', 'you', 'will', 'only', 'have', 'interactions', 'with', 'the', 'family', 'on', 'the', 'shared', 'stairs', 'and', 'stoop', 'there', 'is', 'however', 'no', 'door', 'at', 'the', 'top', 'of', 'the', 'stairs', 'so', 'sounds', 'travel', 'from', 'the', 'rest', 'of', 'the', 'house', 'lisa_roxanne', 'was', 'kind', 'and', 'accommodating', 'and', 'the', 'children', 'are', 'adorable']


In [38]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [39]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['good', 'thing', 'place', 'location', 'right', 'middle', 'clinton_hill', 'easy', 'access', 'restaurant', 'franklin_avenue', 'fort_greene', 'also', 'close', 'train', 'transportation', 'easy', 'space', 'clean', 'neat', 'spare', 'really', 'much', 'floor', 'apartment', 'private', 'room', 'interaction', 'family', 'share', 'stoop', 'however', 'top', 'stair', 'sound', 'kind', 'accommodate', 'child', 'adorable']]


In [40]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 2), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1)]]


In [41]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [42]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.254*"accommodate" + 0.191*"communication" + 0.060*"detail" + 0.057*"wife" '
  '+ 0.054*"bad" + 0.048*"request" + 0.046*"clear" + 0.044*"chance" + '
  '0.042*"tourist" + 0.038*"thoughtful"'),
 (1,
  '0.263*"right" + 0.161*"plenty" + 0.117*"walking_distance" + 0.054*"front" + '
  '0.050*"stair" + 0.044*"middle" + 0.043*"transportation" + 0.039*"top" + '
  '0.038*"child" + 0.038*"adorable"'),
 (2,
  '0.000*"expresstrain" + 0.000*"planta" + 0.000*"מהמם" + 0.000*"מומלץ" + '
  '0.000*"additionaly" + 0.000*"hanne" + 0.000*"אטמים" + 0.000*"komfortabel" + '
  '0.000*"belge" + 0.000*"resolviendo"'),
 (3,
  '0.336*"next" + 0.240*"book" + 0.072*"accommodation" + 0.070*"totally" + '
  '0.046*"hospitable" + 0.031*"bath" + 0.029*"possibly" + 0.025*"story" + '
  '0.018*"accept" + 0.016*"twice"'),
 (4,
  '0.313*"enjoy" + 0.102*"long" + 0.090*"explore" + 0.074*"cute" + 0.054*"end" '
  '+ 0.054*"description" + 0.049*"several" + 0.045*"sheet" + 0.042*"message" + '
  '0.033*"confortable"'),
 (5,


In [43]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -12.554969641272873

Coherence Score:  0.4258117218330876


In [44]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

/Users/tristandealwis/opt/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
15     0.256645 -0.264410       1        1  34.055717
11     0.273911 -0.197524       2        1  31.818367
10     0.138073  0.217810       3        1   5.053983
9      0.129703  0.194478       4        1   4.773134
14     0.083175  0.102526       5        1   3.346077
5      0.087117  0.108615       6        1   3.156082
7      0.080154  0.097255       7        1   2.961296
13     0.006046  0.020619       8        1   1.971589
8     -0.009364  0.009536       9        1   1.774037
19    -0.011993  0.007390      10        1   1.734841
4     -0.016343  0.004493      11        1   1.659154
1     -0.050437 -0.014440      12        1   1.249080
0     -0.074980 -0.025567      13        1   1.135123
18    -0.088705 -0.030926      14        1   1.090243
17    -0.041486 -0.009837      15        1   1.073051
16    -0.095640 -0.033136      16        1   0.979705
3     -0.107326 -0.037199      17        1   0.946606
12    -0.137878 -0.045814      18        1   0.643178
6     -0.178950 -0.053770      19        1   0.571374
2     -0.241721 -0.050099      20        1   0.007375, topic_info=      Category          Freq                  Term         Total  loglift  \
52     Default  84929.000000                  stay  84929.000000  30.0000   
47     Default  80753.000000                 great  80753.000000  29.0000   
4      Default  72437.000000             apartment  72437.000000  28.0000   
22     Default  64517.000000                 place  64517.000000  27.0000   
267    Default  18174.000000                  home  18174.000000  26.0000   
...        ...           ...                   ...           ...      ...   
10774  Topic20      0.005487             correctos      1.290073   4.0548   
10775  Topic20      0.005487  perfekte_gastgeberin      1.290073   4.0548   
10776  Topic20      0.005487                  boas      1.290076   4.0548   
10777  Topic20      0.005487              cartinha      1.290073   4.0548   
10778  Topic20      0.005487                 chave      1.290073   4.0548   

       logprob  
52     30.0000  
47     29.0000  
4      28.0000  
22     27.0000  
267    26.0000  
...        ...  
10774 -10.3822  
10775 -10.3822  
10776 -10.3822  
10777 -10.3822  
10778 -10.3822  

[1662 rows x 6 columns], token_table=      Topic      Freq                Term
term                                     
484       2  0.999718          absolutely
132      17  0.996621              accept
473       6  0.999039          accessible
1        13  0.999816         accommodate
292      17  0.999547       accommodation
...     ...       ...                 ...
442      19  0.998182             willing
280       7  0.995623  without_hesitation
194      16  0.999860           wonderful
368       4  0.999793                work
169      13  0.998107               worry

[366 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[16, 12, 11, 10, 15, 6, 8, 14, 9, 20, 5, 2, 1, 19, 18, 17, 4, 13, 7, 3])